In [1]:
import tensorflow as tf

E:\Anaconda3\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
x = tf.Variable(tf.random_normal([1, 2], stddev=1, seed=1))
w = tf.Variable(tf.random_normal([2, 1], stddev=1, seed=1))
y_ = tf.Variable(tf.random_normal([1, 1], stddev=1, seed=1))
y = tf.matmul(x, w)

loss = tf.reduce_mean(tf.square(y_ - y)) + tf.contrib.layers.l2_regularizer(.5)(w)
init = tf.initialize_all_variables()
with tf.Session() as sess:
    sess.run(init)
    print(sess.run(loss))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
14.210822


In [3]:
# import tensorflow as tf

weight = tf.constant([[1.0, -2.0], [-3.0, 4.0]])
with tf.Session() as sess:
    # 输出为(|1| + |-2| + |-3| + |-4|) * 0.5 = 5 其中0.5 为正则化权重
    print(sess.run(tf.contrib.layers.l1_regularizer(.5)(weight)))
    # 输出为(|1|^2 + |-2|^2 + |-3|^2 + |-4|^2)/2 * 0.5 = 7.5 其中0.5 为正则化权重
    print(sess.run(tf.contrib.layers.l2_regularizer(.5)(weight)))

5.0
7.5


In [4]:
# Use random uniform values in [0, 1) as the initializer for a variable of shape
# [2, 3]. The default type is float32.
var = tf.Variable(tf.random_uniform([2, 3]), name="var")
init = tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init)
    print(sess.run(var))

[[0.65192723 0.7772008  0.23142707]
 [0.9088253  0.29260433 0.02562785]]


In [5]:
# -0.5到0.5均匀分布的3*3数组
var = tf.Variable(tf.random_uniform([3, 3], minval=-0.5, maxval=0.5), name="var")
init = tf.initialize_all_variables()

with tf.Session() as sess:
    sess.run(init)
    print(sess.run(var))

[[ 0.20135999 -0.25651252 -0.21664107]
 [ 0.2578994  -0.47277653 -0.31005192]
 [ 0.26792276 -0.29660547 -0.34434402]]


In [6]:
bias = tf.Variable(tf.constant(0.1, shape=[5]))
init = tf.initialize_all_variables()
with tf.Session() as sess:
    sess.run(init)
    print(sess.run(bias))

[0.1 0.1 0.1 0.1 0.1]


In [7]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

data = []
label = []
np.random.seed(0)

# 以原点为圆心，半径为1的圆把散点划分成红蓝两部分，并加入随机噪音。
for i in range(150):
    x1 = np.random.uniform(-1, 1)
    x2 = np.random.uniform(0, 2)
    if x1 ** 2 + x2 ** 2 <= 1:
        data.append([np.random.normal(x1, 0.1), np.random.normal(x2, 0.1)])
        label.append(0)
    else:
        data.append([np.random.normal(x1, 0.1), np.random.normal(x2, 0.1)])
        label.append(1)
# reshape(-1,1) Numpy自动计算行数量
data = np.hstack(data).reshape(-1, 2)
label = np.hstack(label).reshape(-1, 1)
plt.scatter(data[:, 0], data[:, 1], c=label,
            cmap="RdBu", vmin=-.2, vmax=1.2, edgecolor="white")
plt.show()


# 获取一层神经网络边上的权重，并将这个权重的L2 正则化损失加入名称为'losses'的集合中
def get_weight(shape, lambda1):
    # 生成一个变量
    var = tf.Variable(tf.random_normal(shape), dtype=tf.float32)
    # add_to_collection 函数将这个新生成的变量的L2正则化损失项加入集合
    # 这个函数的第一个参数'losses' 是集合的名字，第二个参数是要加入集合的内容
    tf.add_to_collection('losses', tf.contrib.layers.l2_regularizer(lambda1)(var))
    return var


x = tf.placeholder(tf.float32, shape=(None, 2))
y_ = tf.placeholder(tf.float32, shape=(None, 1))
sample_size = len(data)
print('sample_size = ', sample_size)
# 每层节点的个数
layer_dimension = [2, 10, 5, 3, 1]
# 神经网络的层数
n_layers = len(layer_dimension)

# 这个变量维护前向传播时最深沉的节点，开始的时候就是输入层
cur_layer = x
# 当前层节点的个数
in_dimension = layer_dimension[0]

# 循环生成网络结构
# 通过一个循环来生成5层全连接的神经网络
for i in range(1, n_layers):
    # layer_dimension[i] 为下一层节点的个数
    out_dimension = layer_dimension[i]
    # 生成当前权重的变量，并将这个变量的L2正则化损失加入计算图上的集合
    weight = get_weight([in_dimension, out_dimension], 0.003)
    # 生成shape(1,10)值为0.1的一维矩阵
    bias = tf.Variable(tf.constant(0.1, shape=[out_dimension]))
    # 使用ReLu激活函数
    # 书上为ReLU 源码为elu
    cur_layer = tf.nn.elu(tf.matmul(cur_layer, weight) + bias)
    in_dimension = layer_dimension[i]

y = cur_layer

# 损失函数的定义。# Loss = 平方差 + reg
mse_loss = tf.reduce_sum(tf.pow(y_ - y, 2)) / sample_size
tf.add_to_collection('losses', mse_loss)
# 列表中的元素相加
loss = tf.add_n(tf.get_collection('losses'))

train_op = tf.train.AdamOptimizer(0.001).minimize(loss)
# 迭代次数
TRAINING_STEPS = 40000

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(TRAINING_STEPS):
        sess.run(train_op, feed_dict={x: data, y_: label})
        if i % 2000 == 0:
            print("After %d steps, loss: %f" % (i, sess.run(loss, feed_dict={x: data, y_: label})))

    # 画出训练后的分割曲线
    xx, yy = np.mgrid[-1:1:.01, 0:2:.01]
#     print('xx =', xx)
#     print('yy =', yy)
    grid = np.c_[xx.ravel(), yy.ravel()]
    probs = sess.run(y, feed_dict={x: grid})
    probs = probs.reshape(xx.shape)

plt.scatter(data[:, 0], data[:, 1], c=label,
            cmap="RdBu", vmin=-.2, vmax=1.2, edgecolor="white")
plt.contour(xx, yy, probs, levels=[.5], cmap="Greys", vmin=0, vmax=.1)
plt.show()

ValueError: c of shape (150, 1) not acceptable as a color sequence for x with size 150, y with size 150